# Using TorchServe to list PyTorch models at scale in AWS Marketplace

TorchServe provides a convenient framework for AWS Marketplace sellers to list their products without writing their own endpoint controllers and handlers. Before the release of TorchServe, if you wanted to list a PyTorch model, you needed to develop custom handlers and build your own docker image, figure out how to make correct API calls in and out of the container network, and solve other ad-hoc problems in developing the model server. TorchServe can simplify this process, and the whole listing process can happen in less than 10 minutes.

## Prerequisites
This solution has the following prerequisites:

- An active AWS account
- IAM roles and policies to access AWS services
  - You need a role with `AmazonSageMakerFullAccess`, `AmazonS3FullAccess` and `AmazonEC2ContainerRegistryFullAccess`
  - For more information, see [Adding and removing IAM identity permissions](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html) in the AWS Identity and Access Management User Guide
- AWS services:
  - [Amazon SageMaker](https://aws.amazon.com/sagemaker/)
  - [Amazon S3](https://aws.amazon.com/s3/)
  - [Amazon ECR](https://aws.amazon.com/ecr/)
  - [AWS Marketplace](https://aws.amazon.com/marketplace)

## Solution overview
The following steps show you how to install TorchServe, create a docker image of TorchServe, create a model archive format (`.mar`) file from a PyTorch data format (`.pth`) file, create a SageMaker model package with the TorchServe docker image and model archive file, and finally validate it and list your product in AWS Marketplace.

### Step 0: (Optional) Update AWS CLI, AWS SDK and Amazon SageMaker SDK

In [ ]:
!pip install --upgrade pip
!pip -q install sagemaker awscli boto3 --upgrade 

### Step 1: Git clone TorchServe and install the model archiver

In [ ]:
!git clone https://github.com/pytorch/serve.git
!pip install serve/model-archiver/

### Step 2: Build a TorchServe docker image and push it to Amazon ECR
#### 1. Create a boto3 session and get the account

In [ ]:
import boto3, time, json
sess    = boto3.Session()
sm      = sess.client('sagemaker')
region  = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')

#### 2. Create an Amazon ECR registry through AWS CLI

In [ ]:
registry_name = 'torchserve-base'
!aws ecr create-repository --repository-name {registry_name}

#### 3. Build the docker image and push it to Amazon ECR

In [ ]:
image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

!docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}

**Remember to scan your docker image in Amazon ECR  after you push the image.**

- Sign in your AWS console and go to Amazon ECR.
- Click the repository you created, select the image and then click Scan to scan your image.
- You will see the Scan status as Complete after you scan the docker image.

![image info](./img/docker_image_scan.png)

**Congratulations! You have created a TorchServe docker image.**

### Step 3: Create a TorchServe model archive with a PyTorch model and upload it to Amazon S3
#### 1. Create a TorchServe archive with a PyTorch model (your own model or a downloaded version)

In this notebook let’s download a [DenseNet-161](https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py) model for demonstration.
You can use your own trained version here instead of downloaded one.

In [ ]:
!wget -q https://download.pytorch.org/models/densenet161-8d451a50.pth
    
model_file_name = 'densenet161'

Covert the `.pth` model file to `.mar` using `torch-model-archiver`

In [ ]:
!torch-model-archiver --model-name {model_file_name} \
--version 1.0 --model-file serve/examples/image_classifier/densenet_161/model.py \
--serialized-file densenet161-8d451a50.pth \
--extra-files serve/examples/image_classifier/index_to_name.json \
--handler image_classifier

!ls *.mar

#### 2. Upload the generated .mar archive file to Amazon S3

SageMaker expects that models are in a `tar.gz` file. You need to convert the `.mar` file to `tar.gz` and then upload the model to your default SageMaker S3 bucket in the models directory.

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session(boto_session=sess)

In [ ]:
bucket_name = sagemaker_session.default_bucket()
prefix = 'torchserve'

!tar cvfz {model_file_name}.tar.gz densenet161.mar
!aws s3 cp {model_file_name}.tar.gz s3://{bucket_name}/{prefix}/models/

### Step 4: Deploy an endpoint and make a prediction using Amazon SageMaker SDK
#### 1. Create a SageMaker model with the TorchServe docker image and model file

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

role = sagemaker.get_execution_role()
model_data = f's3://{bucket_name}/{prefix}/models/{model_file_name}.tar.gz'
sm_model_name = 'torchserve-densenet161'

torchserve_model = Model(model_data = model_data, 
                         image_uri = image,
                         role  = role,
                         predictor_cls=Predictor,
                         name  = sm_model_name)

#### 2. Deploy an endpoint with the SageMaker model that you created

In [ ]:
endpoint_name = 'torchserve-endpoint-' + sm_model_name + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

predictor = torchserve_model.deploy(instance_type='ml.m4.xlarge',
                                    initial_instance_count=1,
                                    endpoint_name = endpoint_name)

#### 3. Test the TorchServe hosted endpoint
Use a public image from Amazon S3 to test the endpoint.

In [ ]:
!wget -q https://s3.amazonaws.com/model-server/inputs/kitten.jpg    
file_name = 'kitten.jpg'
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = payload
    
response = predictor.predict(data=payload)
print(*json.loads(response), sep = '\n')

#### 4. Delete the endpoint
To avoid unnecessary billing, delete the endpoint that you created.

In [ ]:
predictor.delete_endpoint()

### Step 5: (Optional) Test the batch transform on SageMaker before listing the PyTorch model in AWS Marketplace

#### 1. Create the batch transform input folder

In [ ]:
sm_model_name = 'torchserve-densenet161'
batch_inference_input_prefix = "batch-inference-input-data"
TRANSFORM_WORKDIR = "transform"

Use two public images from Amazon S3 to test the transform job.

In [ ]:
%%sh

# mkdir transform
cd transform
wget https://s3.amazonaws.com/model-server/inputs/kitten.jpg
wget https://s3.amazonaws.com/model-server/inputs/flower.jpg  

#### 2. Upload the batch transform input folder to an Amazon S3 bucket

In [ ]:
transform_input = sagemaker_session.upload_data(TRANSFORM_WORKDIR, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

#### 3. Create the batch transform job in SageMaker

In [ ]:
transformer = sagemaker.transformer.Transformer(model_name=sm_model_name, instance_count=1, instance_type='ml.m4.xlarge',
                            strategy=None, assemble_with=None, output_path=None, sagemaker_session=sagemaker_session)

In [ ]:
transformer.transform(transform_input, content_type='image/jpeg')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

**Congratulations! The batch transform succeeded.**

### Step 6: Create the model package
Now you can start creating your own model package for listing. Before you create it, you need to specify several fields for [inference specification](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_InferenceSpecification.html) and [model package validation specification](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ModelPackageValidationSpecification.html). Creating the model package also does the validation job.

#### 1. Create the model package inference specification
Specify several fields in the pre-defined inference specification template that I provide.

In [ ]:
from src.inference_specification import InferenceSpecification
import json

modelpackage_inference_specification = InferenceSpecification().get_inference_specification_dict(
    ecr_image=image,
    supports_gpu=True,
    supported_content_types=["image/jpeg", "image/png"],
    supported_mime_types=["application/json"])

# Specify the model data resulting from the previously completed training job
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]= model_data
print(json.dumps(modelpackage_inference_specification, indent=4, sort_keys=True))

#### 2. Create the model package validation specification
Specify several fields in the pre-defined model package validation specification template that I provide.

In [ ]:
from src.modelpackage_validation_specification import ModelPackageValidationSpecification
import time

modelpackage_validation_specification = ModelPackageValidationSpecification().get_validation_specification_dict(
    validation_role = role,
    batch_transform_input = transform_input,
    input_content_type = "image/jpeg",
    output_content_type = "application/json",
    instance_type = "ml.c4.xlarge",
    output_s3_location = 's3://{}/{}'.format(sagemaker_session.default_bucket(), "/batch-inference-output-data"))

print(json.dumps(modelpackage_validation_specification, indent=4, sort_keys=True))

#### 3. Create the model package with inference specification and validation specification

In [ ]:
model_package_name = sm_model_name + "-" + str(round(time.time()))
create_model_package_input_dict = {
    "ModelPackageName" : model_package_name,
    "ModelPackageDescription" : "Model of pre-trained DenseNet161",
    "CertifyForMarketplace" : True
}
create_model_package_input_dict.update(modelpackage_inference_specification)
create_model_package_input_dict.update(modelpackage_validation_specification)
print(json.dumps(create_model_package_input_dict, indent=4, sort_keys=True))

sm.create_model_package(**create_model_package_input_dict)

Creating the model package is an asynchronous process. To check its status, run the following command.

In [ ]:
while True:
    response = sm.describe_model_package(ModelPackageName=model_package_name)
    status = response["ModelPackageStatus"]
    print (status)
    if (status == "Completed" or status == "Failed"):
        print (response["ModelPackageStatusDetails"])
        break
    time.sleep(100)

**Congratulations! You have created a model package for listing in AWS Marketplace.**

### Step 7: (Optional) Create another model package

This step demonstrates that the same TorchServer docker image works for different PyTorch model packages. Download a [VGG-11](https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py) model and create a corresponding model package. You can also use your own model.

#### 1. Create the .mar file and upload to Amazon S3

In [ ]:
!wget -q https://download.pytorch.org/models/vgg11-bbd30ac9.pth
    
model_file_name_vgg11 = 'vgg11'

!torch-model-archiver --model-name {model_file_name_vgg11} \
--version 1.0 --model-file serve/examples/image_classifier/vgg_11/model.py \
--serialized-file vgg11-bbd30ac9.pth \
--extra-files serve/examples/image_classifier/index_to_name.json \
--handler image_classifier

!ls *.mar

In [ ]:
prefix = 'torchserve'

!tar cvfz {model_file_name_vgg11}.tar.gz vgg11.mar
!aws s3 cp {model_file_name_vgg11}.tar.gz s3://{bucket_name}/{prefix}/models/

In [ ]:
model_data = f's3://{bucket_name}/{prefix}/models/{model_file_name_vgg11}.tar.gz'
sm_model_name_vgg11 = 'torchserve-vgg11'

#### 2. Create a new model and its corresponding inference specification and validation specification

In [ ]:
modelpackage_inference_specification = InferenceSpecification().get_inference_specification_dict(
    ecr_image=image,
    supports_gpu=True,
    supported_content_types=["image/jpeg", "image/png"],
    supported_mime_types=["application/json"])

# Specify the model data resulting from the previously completed training job
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]= model_data
print(json.dumps(modelpackage_inference_specification, indent=4, sort_keys=True))


modelpackage_validation_specification = ModelPackageValidationSpecification().get_validation_specification_dict(
    validation_role = role,
    batch_transform_input = transform_input,
    input_content_type = "image/jpeg",
    output_content_type = "application/json",
    instance_type = "ml.c4.xlarge",
    output_s3_location = 's3://{}/{}'.format(sagemaker_session.default_bucket(), "/batch-inference-output-data"))

print(json.dumps(modelpackage_validation_specification, indent=4, sort_keys=True))

#### 3. Create the model package with inference specification and validation specification

In [ ]:
model_package_name = sm_model_name_vgg11 + "-" + str(round(time.time()))
create_model_package_input_dict = {
    "ModelPackageName" : model_package_name,
    "ModelPackageDescription" : "Model of pre-trained VGG11",
    "CertifyForMarketplace" : True
}
create_model_package_input_dict.update(modelpackage_inference_specification)
create_model_package_input_dict.update(modelpackage_validation_specification)
print(json.dumps(create_model_package_input_dict, indent=4, sort_keys=True))

sm.create_model_package(**create_model_package_input_dict)

To check the creation status, run the following code.

In [ ]:
while True:
    response = sm.describe_model_package(ModelPackageName=model_package_name)
    status = response["ModelPackageStatus"]
    print (status)
    if (status == "Completed" or status == "Failed"):
        print (response["ModelPackageStatusDetails"])
        break
    time.sleep(100)

**Congratulations! You have created another model package for listing.**

### Step 8: List your model package in AWS Marketplace management portal

After you created your model packages, they appear on the SageMaker console. Go to SageMaker console, click model packages on the left panel and see the model packages you just created. Select a model package and choose `Publish new ML Marketplace listing`. You’re redirected to [AWS Marketplace Management Portal](https://aws.amazon.com/marketplace/management/ml-products). To start publishing your first PyTorch model, follow the instructions in the Management Portal. For more information, see [Machine learning products](https://docs.aws.amazon.com/marketplace/latest/userguide/machine-learning-products.html) in the AWS Marketplace Seller Guide.

![image info](./img/listing_on_marketplace.png) 

# Conclusion
In this notebook, I showed you how to use TorchServe to create two model package listings (with the same TorchServe docker image) for AWS Marketplace. TorchServe provides a convenient and flexible way to host an endpoint for PyTorch models. TorchServe supports a variety of default [torch handlers](https://github.com/pytorch/serve/tree/master/ts/torch_handler). You can also write your own handler to better support your unique model and then update your docker image.